# Lab 3 - How to improve your results?

In this lab session we will look at data augmentation and debugging strategies.

## Objectives:

1. Learn how to perform data augmentation in tensorflow
2. Experiment with different types of data augmentation
3. Learn how to debug with tensorflow 

## CIFAR10 Dataset prep

The server hosting CIFAR10, the dataset we're using, does not respond well to everyone simultaneously downloading the dataset from our IP address range so instead of downloading the dataset for each lab we've modified the `cifar10` dataset code to store the dataset in a folder outside of the current lab sheet: `~/.cache/adl/cifar-10`

Look in your previous lab sheet folders and find one which has a `./dataset/cifar-10-batches-py` directory, then run the following commands in that labsheet folder (e.g. `Lab_1_intro`):

```
mkdir -p ~/.cache/adl/cifar-10
cp -r ./dataset/cifar-10-batches-py ~/.cache/adl/cifar-10/
```

## Changes to code from Lab 2

We've taken a modified version of the code we've given you for lab 2, instead of using ops defined in the `tf.nn` module to build the network we move to using ops from the `tf.layers` module which is a higher level interface making it easier to try new architectures. We started with `tf.nn` to show you the nuts and bolts of neural networks so nothing was hidden. The `tf.layers` ops are defined in a very similar way to the layers you used in the previous lab, except that they are parameterised so you don't have to repeat the same code over and over again.

We've also improved a few things:

* You can now see the accuracy and loss for both the training and validation sets in tensorboard
* Summaries are flushed periodically so you don't have to wait until the network finishes training to inspect the summaries in tensorboard

Both improvements will prove useful for your second coursework.

The architecture of the network stays the same from the last lab: two convolutional layers followed by two fully connected layers with batch normalisation on the convolutional layers.

**Take a moment to look through the new code cifar_model.py**

## Setup (REMINDER)

1. Login to BC4

    ```
    ssh <your_UoB_ID>@bc4login.acrc.bris.ac.uk
    ```
    
2. Clone the repository

    ```
    git clone "https://github.com/COMSM0018-Applied-Deep-Learning/labsheets.git" ~/labsheets
    ```

3. Change to the lab 3 directory:

    ```
    cd ~/labsheets/Lab_3_Improve/
    ```
    
4. Make all ```go_interactive.sh``` and ```tensorboard_params.sh``` files executables by using the command `chmod`: 

    ```
    chmod +x go_interactive.sh tensorboard_params.sh
    ```
   
5. Switch to interactive mode, and note the change of the gpu login to a reserved gpu:

    ```
    ./go_interactive.sh 
    ```
    
6. Run the following script. It will pop up two values: `ipnport=XXXXX` and `ipnip=XX.XXX.X.X.`

    ```
    ./tensorboard_params.sh
    ```
    
    **Write them down since we will use them for using TensorBoard.**

7. Train the model using the command:
    
    ```
    python cifar_model.py
    ```
   
8. Open a **new terminal window** and login using SSH like in step 1 then run:

    ```
    tensorboard --logdir=logs/ --port=<ipnport>
    ```
    
9. Open a **new terminal window** on your machine and type: 
    
    ```
    ssh -N <USER_NAME>@bc4login.acrc.bris.ac.uk -L 6006:<ipnip>:<ipnport>
    ```

10. Open your web browser (Use Chrome; Firefox currently has issues with tensorboard) and open the port 6006 (http://localhost:6006). This should open TensorBoard, and you can navigate through the summaries that we included.



## 3.1  Data Augmentation

Generally the more data a CNN (or any deep learning model) has access to the better features it learns, and therefore the better it performs. Data augmentation refers to techniques to artificially increase the amount of training data with label preserving transformations on the original training data, i.e. we want to transform CIFAR-10 images in such a way that the data becomes more varied but the object in the image remains the same.

It is typical to implement data augmentation *online* where each training mini-batch is loaded and mutated stocastically in some way (e.g. rotation, translation, blurring). Recall that we don't stop training after having processed the full dataset but after some other stopping criteria and so it is possible, even probable, that we will process each training example more than once. If we mutate the inputs stochastically then each time an example is part of a mini-batch it will be mutated in some different way (e.g. rotating by an angle sampled from a random distribution).

### Practical 3.1: Data Augmentation with Random Flips

One example of data augmentation which can improve results on the CIFAR-10 dataset is horizontal flips. By randomly flipping the image you are able to add additional training data, without making the object in the image unrecognisable. 

* **Q. Think of problems that are invariant to horizontal flips... then think of problems that are not invariant (i.e. a horizontal flip will change the correct label of your sample).**

Implement data augmentation into your network using online data augmentation and retrain to see the improvement in your results. Be careful that you **don't** apply the data augmentation during **testing**.

* **Hint**: For types of data augmentation implemented in tensorflow have a look at the [tf.image](https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/image) module.
* **Hint**: Because the stochastic `tf.image` ops are defined for 3D tensors you'll need to [map](https://www.tensorflow.org/api_docs/python/tf/map_fn) the op over the mini-batch so it applied to each training example.

### Practical 3.2: Your own data augmentation

Think carefully about another form of data augmentation which you believe would improve the capability of your network to recognise images from the CIFAR-10 dataset. Some types of data augmentation may have a negative impact on your network. For instance, flipping the image vertically is not useful as the test data will not contain any upside down boats or cats etc.

Implement your chosen data augmentation method to see if it does improve your results.

**Hint:** If you're stuck for ideas revisit the practical slides or talk to the TAs.

Train your model and save the relevant logs, as well as the code you've written

```
Lab_3_<username>.zip
 |----------cifar_model.py
 |----------logs\ 
 ```



## 3.2 Debugging Strategies

In Labs 1 and 2 you have seen how to use tensorflow to view how the accuracy and loss change over time. This is one of the most valuable tools for gaining insight into what your network is doing and debugging it.

One thing to bear in mind when debugging tensorflow is that you *can't* easily use a normal python `print` statement. This is because tensorflow works by first building a computational graph and then evaluating the graph. You cannot print a value until it has been evaluated in the graph. To add a print operation to a tensorflow graph you use `tf.Print`. For example:

```python
x = tf.get_variable('x', [10, 5])
x = tf.Print(x, [x])

sess = tf.InteractiveSession()
sess.run(x)
```

However, it is generally better pratice and faster to use a debugger. We will go through how to use the tensorflow debugger in this next exercise.


### Practical 3.3: Using the tensorflow debugger

Another useful tools is the [tensorflow debugger](https://www.tensorflow.org/programmers_guide/debugger).

To use it you'll first need to access an interactive node on bluecrystal. To do this run the command:

```bash
./go_interactive.sh

```

Try running the debug example within tensorflow: 

```bash
module add libs/tensorflow/1.2
python -m tensorflow.python.debug.examples.debug_mnist
```

This code also trains a classifier for [MNIST dataset](http://yann.lecun.com/exdb/mnist/), you’ll notice that unlike the previous examples we have trained, the accuracy decreases after step 1 and does not increase. This
is most likely due to a bad numeric value such as `inf` or `nan` being generated in the training graph. The way tfdbg
works is to add filters for tensor values so we can find the problem. Since `inf` and `nan` are common problems this filter already exists.

To debug the example with tfdbg run:

```bash
python −m tensorflow.python.debug.examples.debug mnist −−debug
```

You will see the following screen:

![tfdbg start](img/tfdbg_screenshot_run_start.png)


You can use **PageUp** / **PageDown** / **Home** / **End** to navigate. If you lack those keys use **Fn + Up** / **Fn + Down** / **Fn + Right** / **Fn + Left**

Run `help` to list the available commands, or alternatively you can refer to the [`tfdbg` cheatsheet](https://www.tensorflow.org/programmers_guide/debugger#tfdbg_cli_frequently-used_commands)

```bash
tfdbg> help
```

Run the filter `has_inf_or_nan` to determine which tensors are contain either `Inf` or `NaN` values:

```bash
tfdbg> run -f has_inf_or_nan
```

You should now see this screen:

![has_inf_or_nan](img/tfdbg_screenshot_run_end_inf_nan.png)

The tensors that match the filter are displayed in chronological order, the tensor at the top `cross_entropy/Log:0` is the one in which `NaN` or `Inf` first appeared, so this is a good place to start our debugging.

To view the value of a tensor click on the underlined tensor name e.g. `cross_entropy/Log:0` or enter the equivalent command:

```bash
tfdbg> pt cross_entropy/Log:0
```

To perform a regex search of tensor values run:

```bash
tfdbg> /(inf|nan)
```

OK, so there are `-Inf` values present in the tensor, how do we determine where they originate from? Well let's determine how this tensor was constructed. Use `node_info --traceback cross_entropy/Log` to determine what op output this tensor.

* **Q:  Which line in the stack trace corresponds to user's code that defines the cross entropy?**

## 3.3 [Optional] Free-form exploration of architecture and hyperparameters (again)

Having considered all options from labs 1-3, including the choice of architectures, learning rate, initialisation, data augmentation ... feel free to play with your code in any way you see fit, and observe the improvement (or deterioration) of both your training and validation errors.

For example:
* Try changing the architecture in some way
* Train for longer
* Apply batch normalisation to the fully connected layers
* Chain data transformations (i.e. multiple transformations one after the other) to produce more varied augmented images.

## Preparing Lab 3 Portfolio

You should by now have the following files, which you can zip under the name `Lab_3_<username>.zip`

Note that we are asking you **this lab** to submit a copy of your modified code with data augmentation
    
From your logs, include only the TensorBoard summaries and remove the checkpoints (model.ckpt-* files)

```
 Lab_3_<username>.zip
 |----------cifar_model.py
 |----------logs\ 
```

Store this zip safely. You will be asked to upload all your labs' portfolio to SAFE after Week 10 - check SAFE for deadline details.

## Further Resources

* [`tfdbg` tutorial](https://www.tensorflow.org/programmers_guide/debugger)
* [Using `tfdbg` for batch run scripts](https://www.tensorflow.org/programmers_guide/debugger#offline_debugging_of_remotely-running_sessions)